In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm

# 데이터 로드 및 전처리 (위의 예제와 동일한 방식으로)
df = pd.read_csv("하이닉스 power (3).csv", thousands=',', encoding='cp949')
df['next_day_return'] = (df['종가'].shift(-1) - df['종가']) / df['종가'] * 100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else 0)
df.dropna(inplace=True)

X = df.drop(['날짜', 'target', 'next_day_return'], axis=1)
y = df['target']
# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=66)

# 랜덤 포레스트 모델을 위한 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [10, 50, 100, 200],  # 생성할 트리의 개수
    'max_depth': [None, 10, 20, 30],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]  # 리프 노드에 있어야 할 최소 샘플 수
}

# GridSearchCV 객체 초기화 및 학습
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=1), param_grid, refit=True, verbose=2, cv=5, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print(f"Best parameters: {grid_search_rf.best_params_}")
best_rf_model = grid_search_rf.best_estimator_

# 최적의 모델로 테스트 데이터 예측
y_pred_rf = best_rf_model.predict(X_test)

# 예측 결과에 대한 평가 지표 출력
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='macro')
recall_rf = recall_score(y_test, y_pred_rf, average='macro')
f1_rf = f1_score(y_test, y_pred_rf, average='macro')

print(f"Accuracy (RF): {accuracy_rf:.4f}")
print(f"Precision (RF): {precision_rf:.4f}")
print(f"Recall (RF): {recall_rf:.4f}")
print(f"F1 Score (RF): {f1_rf:.4f}")

Fitting 5 folds for each of 144 candidates, totalling 720 fits


KeyboardInterrupt: 